# Banking Data Quality & Reconciliation Analysis
## A Comprehensive Learning Project

**Objective**: Learn and demonstrate data quality assessment, reconciliation, and business intelligence using real banking data.

**What You'll Learn:**
- Data exploration and profiling techniques
- Comprehensive data quality checks
- Reconciliation methodologies
- Customer analytics and segmentation
- Data preparation for Power BI dashboards

**Dataset Sources:**
1. [Financial Transactions Dataset](https://www.kaggle.com/datasets/cankatsrc/financial-transactions-dataset)
2. [Banking & Customer Transaction Data](https://www.kaggle.com/datasets/yogeshtekawade/banking-and-customer-transaction-data)
3. [Transaction Data for Banking Operations](https://www.kaggle.com/datasets/ziya07/transaction-data-for-banking-operations)

**My Approach**: This notebook shows my actual working process - I import libraries as I need them, work through problems step by step, and document my learning along the way. This is how I'd approach a real data analysis project.

---

## Part 1: Setup & Data Loading

Let's start by loading the data. I'll import libraries as needed throughout the project.

In [ ]:
# First, I need pandas to work with data and pathlib for file handling
import pandas as pd
from pathlib import Path

# Configure pandas to show more readable output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✓ Basic libraries loaded!")
print(f"pandas version: {pd.__version__}")

### Load Customer Data

**Learning Note**: Always start by understanding your master data (customers, products, etc.) before diving into transactional data.

In [ ]:
# Set up project paths
project_root = Path.cwd()
datasets_path = project_root / 'Datasets'
outputs_path = project_root / 'outputs'

# Create outputs directory if it doesn't exist
outputs_path.mkdir(exist_ok=True)

# Load customer data
# REPLACE 'customer_data.csv' with your actual filename
try:
    customer_df = pd.read_csv(datasets_path / 'customer_data.csv')
    print(f"✓ Loaded {len(customer_df):,} customer records")
    print(f"✓ Columns: {list(customer_df.columns)}")
except FileNotFoundError:
    print("⚠ Customer file not found. Please download and place in Datasets/ folder")
    print("Creating sample data for demonstration...")
    
    # For learning purposes, I'll create sample data
    # In real work, you'd use actual datasets
    import numpy as np
    
    customer_df = pd.DataFrame({
        'customer_id': range(1, 101),
        'name': [f'Customer_{i}' for i in range(1, 101)],
        'age': np.random.randint(18, 70, 100),
        'city': np.random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston'], 100),
        'account_type': np.random.choice(['Savings', 'Checking', 'Premium'], 100),
        'signup_date': pd.date_range('2020-01-01', periods=100, freq='3D')
    })
    print(f"✓ Created sample dataset with {len(customer_df):,} records")

# Display first few rows
customer_df.head()

### Load Transaction Data

**Learning Note**: Transaction data is typically much larger than master data. Pay attention to memory usage and data types.

In [ ]:
# Load transaction data
# REPLACE 'transactions.csv' with your actual filename
try:
    transaction_df = pd.read_csv(datasets_path / 'transactions.csv')
    print(f"✓ Loaded {len(transaction_df):,} transaction records")
    print(f"✓ Columns: {list(transaction_df.columns)}")
except FileNotFoundError:
    print("⚠ Transaction file not found. Creating sample data for demonstration...")
    
    # Need numpy for random data generation
    import numpy as np
    
    # Create sample transaction data
    n_transactions = 5000
    transaction_df = pd.DataFrame({
        'transaction_id': range(1, n_transactions + 1),
        'customer_id': np.random.choice(customer_df['customer_id'], n_transactions),
        'transaction_date': pd.date_range('2023-01-01', periods=n_transactions, freq='h'),
        'amount': np.random.uniform(10, 5000, n_transactions).round(2),
        'transaction_type': np.random.choice(['Deposit', 'Withdrawal', 'Transfer', 'Payment'], n_transactions),
        'status': np.random.choice(['Completed', 'Completed', 'Completed', 'Pending', 'Failed'], n_transactions)
    })
    print(f"✓ Created sample dataset with {len(transaction_df):,} records")

# Display first few rows
transaction_df.head()

### Initial Data Profiling

**What I'm Learning**: Before any analysis, understand the structure, size, and basic statistics of your data.

In [ ]:
def profile_dataset(df, name):
    """Generate a comprehensive profile of a dataset"""
    print(f"\n{'='*60}")
    print(f"PROFILE: {name}")
    print(f"{'='*60}")
    
    # Basic info
    print(f"\n📊 Dataset Overview:")
    print(f"   Rows: {len(df):,}")
    print(f"   Columns: {len(df.columns)}")
    print(f"   Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Data types
    print(f"\n📋 Column Data Types:")
    print(df.dtypes.value_counts())
    
    # Missing values
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(f"\n⚠ Missing Values:")
        print(missing[missing > 0].sort_values(ascending=False))
    else:
        print(f"\n✓ No missing values found")
    
    # Duplicates
    dup_count = df.duplicated().sum()
    print(f"\n🔍 Duplicates: {dup_count:,} ({dup_count/len(df)*100:.2f}%)")
    
    return df.info()

# Profile both datasets
profile_dataset(customer_df, "Customer Data")
profile_dataset(transaction_df, "Transaction Data")

---
## Part 2: Data Quality Assessment

**Learning Objective**: Implement comprehensive data quality checks following industry best practices.

**Quality Dimensions:**
1. **Completeness** - Are all required fields populated?
2. **Uniqueness** - Are there duplicates where there shouldn't be?
3. **Validity** - Do values conform to expected formats/ranges?
4. **Consistency** - Are related fields logically consistent?
5. **Accuracy** - Are there outliers or suspicious values?
6. **Integrity** - Do foreign keys match primary keys?

### 2.1 Completeness Check

**What I'm Learning**: Measuring how complete your data is across all fields.

In [ ]:
def check_completeness(df, dataset_name):
    """Analyze data completeness across all columns"""
    
    completeness_results = []
    
    for col in df.columns:
        total_rows = len(df)
        null_count = df[col].isnull().sum()
        non_null_count = total_rows - null_count
        completeness_pct = (non_null_count / total_rows) * 100
        
        # Check for empty strings in object columns
        empty_count = 0
        if df[col].dtype == 'object':
            empty_count = (df[col] == '').sum()
        
        completeness_results.append({
            'Column': col,
            'Total_Rows': total_rows,
            'Non_Null': non_null_count,
            'Null_Count': null_count,
            'Empty_Strings': empty_count,
            'Completeness_%': round(completeness_pct, 2),
            'Status': '✓ Good' if completeness_pct >= 95 else ('⚠ Fair' if completeness_pct >= 80 else '✗ Poor')
        })
    
    completeness_df = pd.DataFrame(completeness_results)
    
    # Summary
    print(f"\n{'='*60}")
    print(f"COMPLETENESS ANALYSIS: {dataset_name}")
    print(f"{'='*60}")
    print(f"\nAverage Completeness: {completeness_df['Completeness_%'].mean():.2f}%")
    
    # Show problematic columns
    issues = completeness_df[completeness_df['Completeness_%'] < 100]
    if len(issues) > 0:
        print(f"\n⚠ Columns with Missing Data:")
        print(issues[['Column', 'Null_Count', 'Completeness_%', 'Status']].to_string(index=False))
    else:
        print("\n✓ All columns are 100% complete!")
    
    return completeness_df

# Check completeness for both datasets
customer_completeness = check_completeness(customer_df, "Customer Data")
transaction_completeness = check_completeness(transaction_df, "Transaction Data")

# Save results
customer_completeness.to_csv(outputs_path / 'quality_customer_completeness.csv', index=False)
transaction_completeness.to_csv(outputs_path / 'quality_transaction_completeness.csv', index=False)
print(f"\n✓ Saved completeness reports to outputs folder")

### 2.2 Uniqueness Check

**What I'm Learning**: Identifying duplicate records that could affect analysis accuracy.

In [ ]:
def check_duplicates(df, dataset_name, key_columns=None):
    """Check for duplicate records"""
    
    print(f"\n{'='*60}")
    print(f"DUPLICATE ANALYSIS: {dataset_name}")
    print(f"{'='*60}")
    
    # Exact duplicates
    exact_dups = df.duplicated().sum()
    print(f"\nExact Duplicates: {exact_dups:,} ({exact_dups/len(df)*100:.2f}%)")
    
    if exact_dups > 0:
        print("\n⚠ Sample of duplicate rows:")
        print(df[df.duplicated(keep=False)].head(10))
    
    # Key column duplicates
    if key_columns:
        for key_col in key_columns:
            if key_col in df.columns:
                key_dups = df.duplicated(subset=[key_col]).sum()
                unique_count = df[key_col].nunique()
                print(f"\nDuplicates in '{key_col}': {key_dups:,}")
                print(f"Unique values: {unique_count:,}")
                
                if key_dups > 0:
                    print(f"⚠ {key_col} should be unique but has duplicates!")
    
    results = {
        'dataset': dataset_name,
        'total_rows': len(df),
        'exact_duplicates': int(exact_dups),
        'duplicate_percentage': round(exact_dups/len(df)*100, 2)
    }
    
    return results

# Check duplicates
customer_dup_results = check_duplicates(customer_df, "Customer Data", ['customer_id'])
transaction_dup_results = check_duplicates(transaction_df, "Transaction Data", ['transaction_id'])

### 2.3 Referential Integrity Check

**What I'm Learning**: Ensuring relationships between tables are valid (foreign keys exist in master tables).

In [ ]:
def check_referential_integrity(transaction_df, customer_df):
    """Validate foreign key relationships"""
    
    print(f"\n{'='*60}")
    print(f"REFERENTIAL INTEGRITY CHECK")
    print(f"{'='*60}")
    
    # Check if customer_id exists in both tables
    if 'customer_id' not in transaction_df.columns or 'customer_id' not in customer_df.columns:
        print("⚠ customer_id column not found in both tables")
        return None
    
    # Get unique customer IDs from each dataset
    txn_customers = set(transaction_df['customer_id'].dropna().unique())
    master_customers = set(customer_df['customer_id'].dropna().unique())
    
    # Find orphaned transactions
    orphaned_ids = txn_customers - master_customers
    orphaned_txn_count = transaction_df[transaction_df['customer_id'].isin(orphaned_ids)].shape[0]
    
    print(f"\nTotal Transactions: {len(transaction_df):,}")
    print(f"Unique Customers in Master: {len(master_customers):,}")
    print(f"Unique Customers in Transactions: {len(txn_customers):,}")
    print(f"\nOrphaned Customer IDs: {len(orphaned_ids):,}")
    print(f"Orphaned Transactions: {orphaned_txn_count:,}")
    
    integrity_score = (1 - orphaned_txn_count/len(transaction_df)) * 100
    print(f"\n{'✓' if integrity_score >= 95 else '⚠'} Referential Integrity Score: {integrity_score:.2f}%")
    
    if orphaned_txn_count > 0:
        print(f"\n⚠ WARNING: {orphaned_txn_count} transactions have no matching customer!")
        print("Sample orphaned customer IDs:", list(orphaned_ids)[:10])
    
    return {
        'total_transactions': len(transaction_df),
        'orphaned_customer_ids': len(orphaned_ids),
        'orphaned_transactions': orphaned_txn_count,
        'integrity_score': round(integrity_score, 2)
    }

# Check integrity
integrity_results = check_referential_integrity(transaction_df, customer_df)

### 2.4 Outlier Detection

**What I'm Learning**: Using statistical methods (IQR) to identify unusual values that might indicate data quality issues.

In [ ]:
# For statistical calculations, I need numpy
import numpy as np

def detect_outliers(df, dataset_name, numeric_columns=None):
    """Detect outliers using IQR method"""
    
    print(f"\n{'='*60}")
    print(f"OUTLIER DETECTION: {dataset_name}")
    print(f"{'='*60}")
    
    if numeric_columns is None:
        numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
    
    outlier_results = []
    
    for col in numeric_columns:
        if col not in df.columns:
            continue
        
        # Calculate IQR
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Find outliers
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outlier_count = len(outliers)
        outlier_pct = (outlier_count / len(df)) * 100
        
        outlier_results.append({
            'Column': col,
            'Mean': round(df[col].mean(), 2),
            'Median': round(df[col].median(), 2),
            'Std': round(df[col].std(), 2),
            'Min': round(df[col].min(), 2),
            'Max': round(df[col].max(), 2),
            'Q1': round(Q1, 2),
            'Q3': round(Q3, 2),
            'Lower_Bound': round(lower_bound, 2),
            'Upper_Bound': round(upper_bound, 2),
            'Outlier_Count': outlier_count,
            'Outlier_%': round(outlier_pct, 2)
        })
        
        print(f"\n{col}:")
        print(f"  Range: [{df[col].min():.2f}, {df[col].max():.2f}]")
        print(f"  Normal Range (IQR): [{lower_bound:.2f}, {upper_bound:.2f}]")
        print(f"  Outliers: {outlier_count} ({outlier_pct:.2f}%)")
    
    outlier_df = pd.DataFrame(outlier_results)
    return outlier_df

# Detect outliers in transaction amounts
amount_cols = [col for col in transaction_df.columns if 'amount' in col.lower() or 'balance' in col.lower()]
if amount_cols:
    outlier_results = detect_outliers(transaction_df, "Transaction Data", amount_cols)
    outlier_results.to_csv(outputs_path / 'quality_outlier_analysis.csv', index=False)
else:
    print("No amount columns found for outlier detection")

### 2.5 Data Quality Summary Dashboard

**What I'm Learning**: Combining multiple quality checks into a single executive summary.

In [ ]:
def create_quality_summary(completeness_df, dup_results, integrity_results, outlier_df=None):
    """Generate overall quality summary"""
    
    summary_data = []
    
    # Completeness
    avg_completeness = completeness_df['Completeness_%'].mean()
    summary_data.append({
        'Quality_Dimension': 'Completeness',
        'Score': round(avg_completeness, 2),
        'Status': '✓ Good' if avg_completeness >= 95 else ('⚠ Fair' if avg_completeness >= 80 else '✗ Poor'),
        'Details': f"{len(completeness_df[completeness_df['Completeness_%'] < 100])} columns incomplete"
    })
    
    # Uniqueness
    dup_score = 100 - dup_results['duplicate_percentage']
    summary_data.append({
        'Quality_Dimension': 'Uniqueness',
        'Score': round(dup_score, 2),
        'Status': '✓ Good' if dup_score >= 95 else ('⚠ Fair' if dup_score >= 90 else '✗ Poor'),
        'Details': f"{dup_results['exact_duplicates']} duplicates found"
    })
    
    # Referential Integrity
    if integrity_results:
        integrity_score = integrity_results['integrity_score']
        summary_data.append({
            'Quality_Dimension': 'Referential Integrity',
            'Score': round(integrity_score, 2),
            'Status': '✓ Good' if integrity_score >= 95 else ('⚠ Fair' if integrity_score >= 85 else '✗ Poor'),
            'Details': f"{integrity_results['orphaned_transactions']} orphaned records"
        })
    
    # Consistency (Outliers)
    if outlier_df is not None and not outlier_df.empty:
        avg_outlier_pct = outlier_df['Outlier_%'].mean()
        outlier_score = 100 - avg_outlier_pct
        summary_data.append({
            'Quality_Dimension': 'Consistency',
            'Score': round(outlier_score, 2),
            'Status': '✓ Good' if outlier_score >= 95 else ('⚠ Fair' if outlier_score >= 85 else '✗ Poor'),
            'Details': f"{avg_outlier_pct:.2f}% outliers on average"
        })
    
    # Overall Score
    overall_score = np.mean([item['Score'] for item in summary_data])
    summary_data.append({
        'Quality_Dimension': '📊 OVERALL QUALITY',
        'Score': round(overall_score, 2),
        'Status': '✓ Good' if overall_score >= 90 else ('⚠ Fair' if overall_score >= 75 else '✗ Poor'),
        'Details': f"Average across {len(summary_data)} dimensions"
    })
    
    summary_df = pd.DataFrame(summary_data)
    
    print(f"\n{'='*70}")
    print(f"DATA QUALITY SUMMARY")
    print(f"{'='*70}\n")
    print(summary_df.to_string(index=False))
    
    return summary_df

# Generate summary
quality_summary = create_quality_summary(
    transaction_completeness, 
    transaction_dup_results, 
    integrity_results,
    outlier_results if 'outlier_results' in locals() else None
)

quality_summary.to_csv(outputs_path / 'data_quality_summary.csv', index=False)
print(f"\n✓ Quality summary saved to outputs folder")

---
## Part 3: Reconciliation Analysis

**Learning Objective**: Implement reconciliation frameworks to identify variances between systems or time periods.

**Key Concepts:**
- System-to-system reconciliation
- Period-over-period analysis
- Variance calculation and thresholds
- Exception management

### 3.1 Period-Over-Period Analysis

**What I'm Learning**: Analyzing trends and detecting unusual changes over time.

In [ ]:
def period_over_period_analysis(df, date_col, amount_col, period='M'):
    """
    Analyze trends and variances over time periods
    
    Parameters:
    - period: 'D' (day), 'W' (week), 'M' (month), 'Q' (quarter), 'Y' (year)
    """
    
    print(f"\n{'='*60}")
    print(f"PERIOD-OVER-PERIOD ANALYSIS (Period: {period})")
    print(f"{'='*60}")
    
    # Ensure date column is datetime
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Create period column
    df_copy = df.copy()
    df_copy['period'] = df_copy[date_col].dt.to_period(period)
    
    # Aggregate by period
    period_summary = df_copy.groupby('period').agg({
        amount_col: ['count', 'sum', 'mean', 'min', 'max']
    }).reset_index()
    
    period_summary.columns = ['Period', 'Transaction_Count', 'Total_Amount', 'Avg_Amount', 'Min_Amount', 'Max_Amount']
    period_summary['Period'] = period_summary['Period'].astype(str)
    
    # Calculate previous period values
    period_summary['Prev_Total_Amount'] = period_summary['Total_Amount'].shift(1)
    period_summary['Prev_Transaction_Count'] = period_summary['Transaction_Count'].shift(1)
    
    # Calculate variances
    period_summary['Amount_Variance'] = period_summary['Total_Amount'] - period_summary['Prev_Total_Amount']
    period_summary['Count_Variance'] = period_summary['Transaction_Count'] - period_summary['Prev_Transaction_Count']
    
    # Calculate percentage changes
    period_summary['Amount_Change_%'] = np.where(
        period_summary['Prev_Total_Amount'] != 0,
        (period_summary['Amount_Variance'] / period_summary['Prev_Total_Amount']) * 100,
        np.nan
    )
    
    period_summary['Count_Change_%'] = np.where(
        period_summary['Prev_Transaction_Count'] != 0,
        (period_summary['Count_Variance'] / period_summary['Prev_Transaction_Count']) * 100,
        np.nan
    )
    
    # Flag significant changes (>10%)
    period_summary['Significant_Change'] = np.where(
        np.abs(period_summary['Amount_Change_%']) > 10,
        '⚠ YES',
        'No'
    )
    
    print(f"\nLast 12 Periods:")
    print(period_summary[['Period', 'Transaction_Count', 'Total_Amount', 
                          'Amount_Change_%', 'Significant_Change']].tail(12).to_string(index=False))
    
    return period_summary

# Perform monthly analysis
if 'transaction_date' in transaction_df.columns and 'amount' in transaction_df.columns:
    monthly_trends = period_over_period_analysis(transaction_df, 'transaction_date', 'amount', 'M')
    monthly_trends.to_csv(outputs_path / 'reconciliation_monthly_trends.csv', index=False)
    print(f"\n✓ Monthly trends saved to outputs folder")
else:
    print("⚠ Required columns not found for period analysis")

### 3.2 System-to-System Reconciliation

**What I'm Learning**: Comparing data from different sources to identify discrepancies.

In [ ]:
def reconcile_systems(source_df, target_df, group_cols, amount_col, 
                      source_name='Source', target_name='Target', tolerance_pct=1):
    """
    Reconcile two datasets by comparing aggregated values
    
    Parameters:
    - tolerance_pct: Acceptable variance threshold (default 1%)
    """
    
    print(f"\n{'='*60}")
    print(f"SYSTEM RECONCILIATION: {source_name} vs {target_name}")
    print(f"{'='*60}")
    
    # Aggregate source
    source_agg = source_df.groupby(group_cols)[amount_col].agg([
        ('count', 'count'),
        ('sum', 'sum'),
        ('mean', 'mean')
    ]).reset_index()
    source_agg.columns = group_cols + [f'{source_name}_Count', f'{source_name}_Sum', f'{source_name}_Avg']
    
    # Aggregate target
    target_agg = target_df.groupby(group_cols)[amount_col].agg([
        ('count', 'count'),
        ('sum', 'sum'),
        ('mean', 'mean')
    ]).reset_index()
    target_agg.columns = group_cols + [f'{target_name}_Count', f'{target_name}_Sum', f'{target_name}_Avg']
    
    # Merge
    recon = source_agg.merge(target_agg, on=group_cols, how='outer', indicator=True)
    recon = recon.fillna(0)
    
    # Calculate variances
    recon['Count_Variance'] = recon[f'{source_name}_Count'] - recon[f'{target_name}_Count']
    recon['Sum_Variance'] = recon[f'{source_name}_Sum'] - recon[f'{target_name}_Sum']
    
    # Variance percentage
    recon['Variance_%'] = np.where(
        recon[f'{target_name}_Sum'] != 0,
        (recon['Sum_Variance'] / recon[f'{target_name}_Sum']) * 100,
        np.where(recon[f'{source_name}_Sum'] != 0, 100, 0)
    )
    
    # Exception flagging
    recon['Exception'] = np.where(
        np.abs(recon['Variance_%']) > tolerance_pct,
        '⚠ EXCEPTION',
        '✓ OK'
    )
    
    # Match status
    recon['Match_Status'] = recon['_merge'].map({
        'both': 'Matched',
        'left_only': f'Only in {source_name}',
        'right_only': f'Only in {target_name}'
    })
    recon = recon.drop('_merge', axis=1)
    
    # Summary
    total_groups = len(recon)
    exceptions = len(recon[recon['Exception'] == '⚠ EXCEPTION'])
    matched = len(recon[recon['Match_Status'] == 'Matched'])
    
    print(f"\nTotal Groups: {total_groups}")
    print(f"Matched: {matched} ({matched/total_groups*100:.2f}%)")
    print(f"Exceptions: {exceptions} ({exceptions/total_groups*100:.2f}%)")
    print(f"\nTotal {source_name}: ${recon[f'{source_name}_Sum'].sum():,.2f}")
    print(f"Total {target_name}: ${recon[f'{target_name}_Sum'].sum():,.2f}")
    print(f"Total Variance: ${recon['Sum_Variance'].sum():,.2f}")
    
    # Show top exceptions
    if exceptions > 0:
        print(f"\n⚠ Top 10 Exceptions by Variance %:")
        top_exceptions = recon[recon['Exception'] == '⚠ EXCEPTION'].sort_values(
            'Variance_%', key=abs, ascending=False
        ).head(10)
        print(top_exceptions[group_cols + ['Variance_%', 'Sum_Variance', 'Match_Status']].to_string(index=False))
    
    return recon

# Example: Simulate two systems by splitting transaction data
# In real scenarios, you'd load from different sources
if len(transaction_df) > 100:
    print("\nSimulating System A vs System B (for demonstration)...")
    
    # Create two "systems" by splitting data or adding small variations
    system_a = transaction_df.sample(frac=0.8, random_state=42)
    system_b = transaction_df.sample(frac=0.8, random_state=123)
    
    # Find a grouping column
    group_col = None
    for col in ['transaction_type', 'type', 'category', 'status']:
        if col in transaction_df.columns:
            group_col = col
            break
    
    if group_col and 'amount' in transaction_df.columns:
        reconciliation = reconcile_systems(
            system_a, system_b, 
            group_cols=[group_col],
            amount_col='amount',
            source_name='System_A',
            target_name='System_B',
            tolerance_pct=1
        )
        
        reconciliation.to_csv(outputs_path / 'reconciliation_system_comparison.csv', index=False)
        print(f"\n✓ Reconciliation saved to outputs folder")
    else:
        print("⚠ Required columns not found for system reconciliation")

---
## Part 4: Customer Analytics & Segmentation

**Learning Objective**: Extract business insights and segment customers based on behavior.

**Key Techniques:**
- RFM Analysis (Recency, Frequency, Monetary)
- Customer segmentation
- Transaction pattern analysis

### 4.1 RFM Analysis

**What I'm Learning**: RFM is a proven method for customer segmentation based on transaction behavior.

In [ ]:
def perform_rfm_analysis(transaction_df, customer_df, reference_date=None):
    """
    Perform RFM (Recency, Frequency, Monetary) Analysis
    """
    
    print(f"\n{'='*60}")
    print(f"RFM ANALYSIS")
    print(f"{'='*60}")
    
    # Check required columns
    if 'customer_id' not in transaction_df.columns or 'amount' not in transaction_df.columns:
        print("⚠ Required columns not found")
        return None
    
    # Set reference date (latest transaction date)
    if reference_date is None:
        if 'transaction_date' in transaction_df.columns:
            transaction_df['transaction_date'] = pd.to_datetime(transaction_df['transaction_date'])
            reference_date = transaction_df['transaction_date'].max()
        else:
            reference_date = pd.Timestamp.now()
    
    print(f"Reference Date: {reference_date.date()}")
    
    # Calculate RFM metrics
    rfm_data = []
    
    for customer_id in transaction_df['customer_id'].unique():
        customer_txns = transaction_df[transaction_df['customer_id'] == customer_id]
        
        # Recency: days since last transaction
        if 'transaction_date' in transaction_df.columns:
            last_txn_date = customer_txns['transaction_date'].max()
            recency = (reference_date - last_txn_date).days
        else:
            recency = 0
        
        # Frequency: number of transactions
        frequency = len(customer_txns)
        
        # Monetary: total transaction value
        monetary = customer_txns['amount'].sum()
        
        rfm_data.append({
            'customer_id': customer_id,
            'Recency': recency,
            'Frequency': frequency,
            'Monetary': monetary
        })
    
    rfm_df = pd.DataFrame(rfm_data)
    
    # Create RFM scores (1-5 scale, 5 being best)
    rfm_df['R_Score'] = pd.qcut(rfm_df['Recency'], q=5, labels=[5, 4, 3, 2, 1], duplicates='drop')
    rfm_df['F_Score'] = pd.qcut(rfm_df['Frequency'], q=5, labels=[1, 2, 3, 4, 5], duplicates='drop')
    rfm_df['M_Score'] = pd.qcut(rfm_df['Monetary'], q=5, labels=[1, 2, 3, 4, 5], duplicates='drop')
    
    # Convert to numeric
    rfm_df['R_Score'] = pd.to_numeric(rfm_df['R_Score'])
    rfm_df['F_Score'] = pd.to_numeric(rfm_df['F_Score'])
    rfm_df['M_Score'] = pd.to_numeric(rfm_df['M_Score'])
    
    # Overall RFM score
    rfm_df['RFM_Score'] = rfm_df['R_Score'] + rfm_df['F_Score'] + rfm_df['M_Score']
    
    # Segment customers
    def segment_customer(row):
        score = row['RFM_Score']
        if score >= 13:
            return 'Champions'
        elif score >= 10:
            return 'Loyal Customers'
        elif score >= 7:
            return 'Potential Loyalists'
        elif score >= 5:
            return 'At Risk'
        else:
            return 'Lost'
    
    rfm_df['Segment'] = rfm_df.apply(segment_customer, axis=1)
    
    # Summary by segment
    segment_summary = rfm_df.groupby('Segment').agg({
        'customer_id': 'count',
        'Recency': 'mean',
        'Frequency': 'mean',
        'Monetary': 'mean'
    }).round(2)
    segment_summary.columns = ['Customer_Count', 'Avg_Recency_Days', 'Avg_Frequency', 'Avg_Monetary']
    
    print(f"\nCustomer Segmentation:")
    print(segment_summary.to_string())
    
    # Merge with customer details
    if not customer_df.empty and 'customer_id' in customer_df.columns:
        rfm_with_details = rfm_df.merge(customer_df, on='customer_id', how='left')
        return rfm_with_details
    
    return rfm_df

# Perform RFM analysis
rfm_results = perform_rfm_analysis(transaction_df, customer_df)

if rfm_results is not None:
    rfm_results.to_csv(outputs_path / 'customer_rfm_analysis.csv', index=False)
    print(f"\n✓ RFM analysis saved to outputs folder")

### 4.2 Transaction Pattern Visualization

**What I'm Learning**: Visual analysis helps identify trends and anomalies quickly.

In [ ]:
# For visualizations, I'll need matplotlib and seaborn
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Set up visualization preferences
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Visualize transaction patterns
if 'transaction_date' in transaction_df.columns and 'amount' in transaction_df.columns:
    
    transaction_df['transaction_date'] = pd.to_datetime(transaction_df['transaction_date'])
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Transaction Analysis Dashboard', fontsize=16, fontweight='bold')
    
    # 1. Daily transaction volume
    daily_txns = transaction_df.groupby(transaction_df['transaction_date'].dt.date)['amount'].agg(['count', 'sum'])
    axes[0, 0].plot(daily_txns.index, daily_txns['count'], marker='o', linewidth=2)
    axes[0, 0].set_title('Daily Transaction Volume')
    axes[0, 0].set_xlabel('Date')
    axes[0, 0].set_ylabel('Transaction Count')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. Amount distribution
    axes[0, 1].hist(transaction_df['amount'], bins=50, edgecolor='black', alpha=0.7)
    axes[0, 1].set_title('Transaction Amount Distribution')
    axes[0, 1].set_xlabel('Amount ($)')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Transaction type breakdown
    if 'transaction_type' in transaction_df.columns:
        type_counts = transaction_df['transaction_type'].value_counts()
        axes[1, 0].pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%', startangle=90)
        axes[1, 0].set_title('Transaction Type Distribution')
    else:
        axes[1, 0].text(0.5, 0.5, 'No transaction type data', ha='center', va='center')
        axes[1, 0].set_title('Transaction Type Distribution')
    
    # 4. Monthly trends
    monthly = transaction_df.groupby(transaction_df['transaction_date'].dt.to_period('M'))['amount'].sum()
    monthly.index = monthly.index.astype(str)
    axes[1, 1].bar(range(len(monthly)), monthly.values, alpha=0.7)
    axes[1, 1].set_title('Monthly Transaction Volume')
    axes[1, 1].set_xlabel('Month')
    axes[1, 1].set_ylabel('Total Amount ($)')
    axes[1, 1].set_xticks(range(len(monthly)))
    axes[1, 1].set_xticklabels(monthly.index, rotation=45)
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(outputs_path / 'transaction_analysis_dashboard.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✓ Dashboard saved to outputs folder")

---
## Part 5: Data Preparation for Power BI

**Learning Objective**: Create clean, aggregated datasets optimized for Power BI dashboards.

In [ ]:
def prepare_powerbi_export():
    """
    Create consolidated datasets for Power BI import
    """
    
    print(f"\n{'='*60}")
    print(f"PREPARING POWER BI EXPORT")
    print(f"{'='*60}")
    
    # Merge customer and transaction data
    if not customer_df.empty and not transaction_df.empty:
        merged_data = transaction_df.merge(
            customer_df,
            on='customer_id',
            how='left',
            suffixes=('_txn', '_cust')
        )
        
        print(f"\n✓ Merged {len(merged_data):,} transaction records with customer data")
        
        # Add calculated fields useful for Power BI
        if 'transaction_date' in merged_data.columns:
            merged_data['transaction_date'] = pd.to_datetime(merged_data['transaction_date'])
            merged_data['Year'] = merged_data['transaction_date'].dt.year
            merged_data['Month'] = merged_data['transaction_date'].dt.month
            merged_data['Month_Name'] = merged_data['transaction_date'].dt.month_name()
            merged_data['Quarter'] = merged_data['transaction_date'].dt.quarter
            merged_data['Day_of_Week'] = merged_data['transaction_date'].dt.day_name()
            merged_data['Is_Weekend'] = merged_data['transaction_date'].dt.dayofweek.isin([5, 6])
        
        # Create summary tables for Power BI
        
        # 1. Customer Summary
        customer_summary = merged_data.groupby('customer_id').agg({
            'amount': ['count', 'sum', 'mean', 'min', 'max'],
            'transaction_date': ['min', 'max']
        }).reset_index()
        customer_summary.columns = ['customer_id', 'Total_Transactions', 'Total_Amount', 
                                    'Avg_Amount', 'Min_Amount', 'Max_Amount',
                                    'First_Transaction', 'Last_Transaction']
        
        # Add customer tenure
        customer_summary['Tenure_Days'] = (
            customer_summary['Last_Transaction'] - customer_summary['First_Transaction']
        ).dt.days
        
        print(f"✓ Created customer summary: {len(customer_summary):,} records")
        
        # 2. Monthly Summary
        monthly_summary = merged_data.groupby(['Year', 'Month', 'Month_Name']).agg({
            'transaction_id': 'count',
            'amount': 'sum',
            'customer_id': 'nunique'
        }).reset_index()
        monthly_summary.columns = ['Year', 'Month', 'Month_Name', 'Transaction_Count', 
                                   'Total_Amount', 'Unique_Customers']
        
        print(f"✓ Created monthly summary: {len(monthly_summary):,} records")
        
        # Export to Excel with multiple sheets
        with pd.ExcelWriter(outputs_path / 'powerbi_export.xlsx', engine='openpyxl') as writer:
            merged_data.to_excel(writer, sheet_name='Transaction_Details', index=False)
            customer_summary.to_excel(writer, sheet_name='Customer_Summary', index=False)
            monthly_summary.to_excel(writer, sheet_name='Monthly_Summary', index=False)
            if rfm_results is not None:
                rfm_results.to_excel(writer, sheet_name='RFM_Segments', index=False)
            quality_summary.to_excel(writer, sheet_name='Data_Quality', index=False)
        
        print(f"\n✓ Power BI export saved: powerbi_export.xlsx")
        print(f"  Contains {merged_data.memory_usage(deep=True).sum() / 1024**2:.2f} MB of data")
        
        return merged_data, customer_summary, monthly_summary
    else:
        print("⚠ Cannot create export: missing customer or transaction data")
        return None, None, None

# Prepare Power BI export
merged_data, customer_summary, monthly_summary = prepare_powerbi_export()

---
## Part 6: Key Insights & Recommendations

**What I Learned**: Always summarize findings and provide actionable recommendations.

In [ ]:
def generate_insights_report():
    """
    Generate a comprehensive insights report
    """
    
    print(f"\n{'='*70}")
    print(f"FINAL INSIGHTS & RECOMMENDATIONS")
    print(f"{'='*70}\n")
    
    insights = []
    
    # Data Quality Insights
    insights.append("📊 DATA QUALITY:")
    avg_quality = quality_summary[quality_summary['Quality_Dimension'] == '📊 OVERALL QUALITY']['Score'].values[0]
    insights.append(f"  • Overall Data Quality Score: {avg_quality:.2f}%")
    
    poor_dimensions = quality_summary[
        (quality_summary['Status'] == '✗ Poor') & 
        (quality_summary['Quality_Dimension'] != '📊 OVERALL QUALITY')
    ]
    if len(poor_dimensions) > 0:
        insights.append(f"  • ⚠ {len(poor_dimensions)} quality dimension(s) need attention")
        for _, row in poor_dimensions.iterrows():
            insights.append(f"    - {row['Quality_Dimension']}: {row['Details']}")
    
    # Transaction Insights
    if not transaction_df.empty:
        insights.append(f"\n💰 TRANSACTION ANALYSIS:")
        insights.append(f"  • Total Transactions: {len(transaction_df):,}")
        if 'amount' in transaction_df.columns:
            total_amount = transaction_df['amount'].sum()
            avg_amount = transaction_df['amount'].mean()
            insights.append(f"  • Total Volume: ${total_amount:,.2f}")
            insights.append(f"  • Average Transaction: ${avg_amount:,.2f}")
    
    # Customer Insights
    if rfm_results is not None:
        insights.append(f"\n👥 CUSTOMER INSIGHTS:")
        segment_counts = rfm_results['Segment'].value_counts()
        insights.append(f"  • Total Customers Analyzed: {len(rfm_results):,}")
        insights.append(f"  • Top Segment: {segment_counts.index[0]} ({segment_counts.values[0]} customers)")
        
        champions = len(rfm_results[rfm_results['Segment'] == 'Champions'])
        at_risk = len(rfm_results[rfm_results['Segment'] == 'At Risk'])
        insights.append(f"  • Champions: {champions} customers")
        if at_risk > 0:
            insights.append(f"  • ⚠ At Risk: {at_risk} customers need retention efforts")
    
    # Recommendations
    insights.append(f"\n✅ RECOMMENDATIONS:")
    insights.append(f"  1. Address data quality issues in poor-scoring dimensions")
    insights.append(f"  2. Implement automated quality monitoring")
    insights.append(f"  3. Set up reconciliation processes for ongoing variance detection")
    if rfm_results is not None and len(rfm_results[rfm_results['Segment'] == 'At Risk']) > 0:
        insights.append(f"  4. Launch retention campaign for 'At Risk' customers")
    insights.append(f"  5. Create Power BI dashboard for stakeholder visibility")
    
    # Print all insights
    for insight in insights:
        print(insight)
    
    # Save to file
    with open(outputs_path / 'insights_report.txt', 'w') as f:
        f.write('\n'.join(insights))
    
    print(f"\n✓ Insights report saved to outputs folder")
    
    return insights

# Generate final report
final_insights = generate_insights_report()

---
## Summary: What I've Accomplished

In this project, I demonstrated:

✅ **Data Exploration & Profiling**
- Loaded and profiled customer and transaction datasets
- Identified data types, structures, and initial quality issues

✅ **Comprehensive Data Quality Assessment**
- Completeness checks across all columns
- Duplicate detection and uniqueness validation
- Referential integrity verification
- Statistical outlier detection using IQR method
- Created quality scorecards and summaries

✅ **Reconciliation Analysis**
- Period-over-period trend analysis
- System-to-system comparison
- Variance calculation and exception flagging
- Automated exception management

✅ **Customer Analytics**
- RFM (Recency, Frequency, Monetary) segmentation
- Customer behavior analysis
- Transaction pattern identification

✅ **Business Intelligence**
- Created visualization dashboards
- Prepared optimized datasets for Power BI
- Generated actionable insights and recommendations

✅ **Professional Documentation**
- Well-commented, production-ready code
- Clear explanations of methodologies
- Organized outputs and reports

### 📁 Output Files Created:
All analysis results saved to the `outputs/` folder for easy sharing and review.

### 🎯 Next Steps:
1. Import `powerbi_export.xlsx` into Power BI
2. Create interactive dashboards
3. Set up automated data pipelines
4. Implement recommendations

---
## 📚 Learning Resources & SQL Integration

**Note**: This notebook focused on Python analysis. See `comprehensive_analysis.sql` for:
- SQL-based data quality checks
- Database reconciliation queries
- Business intelligence SQL scripts
- Performance optimization techniques

**Remember**: 
- Python is great for complex analysis and machine learning
- SQL is essential for database operations and large-scale data processing
- Both skills are crucial for data analytics roles!